In [15]:
import numpy as np
import pandas as pd
import keras
import h5py
import os
import cv2
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers import Conv2D,Flatten,Dropout,Activation,Dense

In [16]:
train_labels=pd.read_csv('labels.csv')
labels_submit=pd.read_csv('sample_submission.csv')
labels_submit.values

array([['000621fb3cbb32d8935728e48679680e', 0.00833333333333333,
        0.00833333333333333, ..., 0.00833333333333333,
        0.00833333333333333, 0.00833333333333333],
       ['00102ee9d8eb90812350685311fe5890', 0.00833333333333333,
        0.00833333333333333, ..., 0.00833333333333333,
        0.00833333333333333, 0.00833333333333333],
       ['0012a730dfa437f5f3613fb75efcd4ce', 0.00833333333333333,
        0.00833333333333333, ..., 0.00833333333333333,
        0.00833333333333333, 0.00833333333333333],
       ...,
       ['fff74b59b758bbbf13a5793182a9bbe4', 0.00833333333333333,
        0.00833333333333333, ..., 0.00833333333333333,
        0.00833333333333333, 0.00833333333333333],
       ['fff7d50d848e8014ac1e9172dc6762a3', 0.00833333333333333,
        0.00833333333333333, ..., 0.00833333333333333,
        0.00833333333333333, 0.00833333333333333],
       ['fffbff22c1f51e3dc80c4bf04089545b', 0.00833333333333333,
        0.00833333333333333, ..., 0.00833333333333333,
        0.008

In [17]:
targets_series = pd.Series(train_labels['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot_labels=np.asarray(one_hot)
one_hot_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [18]:
im_size = 224


x_train = []
y_train = []


i = 0 
for f, breed in train_labels.values:
    if(i<1000):
        img = cv2.imread('train/{}.jpg'.format(f))
        label = one_hot_labels[i]
        x_train.append(cv2.resize(img, (im_size, im_size)))
        y_train.append(label)
        i += 1
    


In [19]:
im_size = 224
x_test = []
j=0
for f in labels_submit['id'].values:
    if(j<100):
        
        img=cv2.imread('test/{}.jpg'.format(f))
        x_test.append(cv2.resize(img,(im_size,im_size)))
        j=j+1


In [20]:
x_train_raw=np.asarray(x_train,np.uint8)/255
y_train_raw=np.asarray(y_train,np.uint8)
x_test_raw=np.asarray(x_test,np.uint8)/255

In [21]:
y_train_raw.shape

(1000, 120)

In [22]:
model = Sequential()
 
model.add(Conv2D(32, 3, 3, activation='relu', input_shape=(224,224,3)))
model.add(Conv2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(120, activation='softmax'))
 
# 8. Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
# 9. Fit model on training data
model.fit(x_train_raw, y_train_raw, 
          batch_size=32, nb_epoch=1, verbose=1)
 

/home/anumeha/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224,...)`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/anumeha/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  after removing the cwd from sys.path.
/home/anumeha/miniconda3/lib/python3.6/site-packages/keras/models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
1000/1000 [==============================] - 199s 199ms/step - loss: 6.8834 - acc: 0.0090


In [23]:
model.predict(x_test_raw, batch_size=None, verbose=0)

array([[0.00831394, 0.00840183, 0.00842199, ..., 0.00836029, 0.00835901,
        0.00832426],
       [0.00831201, 0.00839691, 0.00841955, ..., 0.008362  , 0.00835138,
        0.00832869],
       [0.00831367, 0.00840059, 0.00842352, ..., 0.00836019, 0.00835882,
        0.00832482],
       ...,
       [0.00831411, 0.00840171, 0.00842193, ..., 0.00836065, 0.00835901,
        0.0083241 ],
       [0.0083144 , 0.00840057, 0.00842152, ..., 0.00836191, 0.00835785,
        0.00832516],
       [0.00831388, 0.00840159, 0.00842198, ..., 0.0083604 , 0.00835912,
        0.00832418]], dtype=float32)